In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=bfbc1e5ad1bd1a977c024fc616f872dd7f8a87bdab0c0a133dbec1343c610e5e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
!pip install --upgrade gspread gspread_dataframe

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:
import pandas as pd
import numpy as np

spreadsheet_key = "1nSywyYPLCol2B4qxaZZq181_7JrrSEA0wx31vNfnd_U" # Extract the key from the URL
sh = gc.open_by_key(spreadsheet_key) # Use open_by_key instead of open
n = 14
for i in range(1,n):
  worksheet = sh.get_worksheet(i)
  import pandas as pd
  from gspread_dataframe import get_as_dataframe

  df = get_as_dataframe(worksheet, evaluate_formulas=True)
  df.dropna(how='all', inplace=True)

  df.to_csv("data.csv", index=False)
  #Eureka Moment!
  import pandas as pd
  import numpy as np
  from ta.volatility import AverageTrueRange
  from ta.trend import MACD
  from ta.utils import dropna

  data = pd.read_csv("/content/data.csv")
  # Indicators
  macd = MACD(data['Close'])
  data['macd'] = macd.macd()
  data['signal_line'] = macd.macd_signal()
  data['macd_diff'] = macd.macd_diff()

  atr = AverageTrueRange(data['High'], data['Low'], data['Close'])
  data['atr'] = atr.average_true_range()

  data['average_volume'] = data['Volume'].rolling(20).mean()

  # Fibo Retract
  data['fib_61.8'] = data['Close'] * 0.618

  # HAC's
  data['ha_close'] = (data['Open'] + data['Close'] + data['High'] + data['Low']) / 4
  data['ha_open'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2
  data['ha_open'] = data['ha_open'].fillna(data['ha_close'])

  window = 20
  data['roll_zscore'] = (data['atr'] - data['atr'].rolling(window).mean()) / data['atr'].rolling(window).std()
  data['roll_zscore_volume'] = (data['Volume'] - data['Volume'].rolling(window).mean()) / data['Volume'].rolling(window).std()



  def calculate_signals(data, atr_threshold=0.015, volume_multiplier=1.5, indicator_threshold=3):
      data['Signal'] = 0
      data['position'] = 0

      current_position = 0

      for i in range(len(data)):
          score = 0

          macd_condition = data['macd_diff'].iloc[i] > 0
          atr_condition = data['atr'].iloc[i] > atr_threshold * data['Close'].iloc[i]
          volume_condition = data['Volume'].iloc[i] > volume_multiplier * data['average_volume'].iloc[i]
          fib_condition = abs(data['Close'].iloc[i] - data['fib_61.8'].iloc[i]) <= 0.01 * data['Close'].iloc[i]
          ha_condition_long = data['ha_close'].iloc[i] > data['ha_open'].iloc[i]
          ha_condition_short = data['ha_close'].iloc[i] < data['ha_open'].iloc[i]

          score += macd_condition
          score += atr_condition
          score += volume_condition
          score += fib_condition
          score += ha_condition_long or ha_condition_short

          raw_signal = 0
          if score >= indicator_threshold:
              if macd_condition and ha_condition_long:
                  raw_signal = 1
              elif macd_condition and ha_condition_short:
                  raw_signal = -1

          # Now decide the actual signal
          signal = 0

          if current_position == 0:
              if raw_signal == 1:
                  signal = 1
                  current_position = 1
              elif raw_signal == -1:
                  signal = -1
                  current_position = -1
          elif current_position == 1:
              if raw_signal == -1:
                  signal = -1  # reverse directly to short
                  current_position = -1
              elif raw_signal != 1:
                  signal = 0  # square off or hold
          elif current_position == -1:
              if raw_signal == 1:
                  signal = 1  # reverse directly to long
                  current_position = 1
              elif raw_signal != -1:
                  signal = 0  # square off or hold


          data.at[i, 'Signal'] = signal
          data.at[i, 'position'] = current_position  # <-- Save position history

      return data


  def clean_repetitive_signals(data):
      previous_signal = 0
      for i in range(len(data)):
          current_signal = data.at[i, 'Signal']


          if current_signal != 0 and current_signal == previous_signal:
              data.at[i, 'Signal'] = 0
          elif current_signal != 0:
              previous_signal = current_signal

      return data



  def generate_tradetype(data):

      data['trade_type'] = data['Signal'].map({
          2: 'long',
          1: 'long',
          0: 'hold',
          -1: 'short',
          -2: 'short'
      })

      return data

  def calculate_dynamic_weights(data, window=20):
    # Calculate success rate of each condition for past trades
    data['macd_success'] = (data['macd_diff'] > 0) & (data['Close'].shift(-5) > data['Close'])
    data['atr_success'] = (-2.5 < data['roll_zscore']) & (data['roll_zscore'] < 2.5) & (data['Close'].shift(-5).abs() > data['Close'].abs())
    data['vol_success'] = (-2.5 < data['roll_zscore_volume']) & (data['roll_zscore_volume'] <2.5)
    data['ha_success'] = (data['ha_close'] > data['ha_open']) & (data['Close'].shift(-5) > data['Close'])

    weights = {
        'macd': data['macd_success'].rolling(window, min_periods=1).mean().fillna(0.5),
        'atr': data['atr_success'].rolling(window, min_periods=1).mean().fillna(0.5),
        'vol': data['vol_success'].rolling(window, min_periods=1).mean().fillna(0.5),
        'ha': data['ha_success'].rolling(window, min_periods=1).mean().fillna(0.5)
    }

    return weights

  def add_weighted_score(data, weights):
    weighted_score = []

    for i in range(len(data)):
        score = 0
        macd_score = (data['macd_diff'].iloc[i] > 0) * weights['macd'].iloc[i]
        atr_score = (data['atr'].iloc[i] > 0.015 * data['Close'].iloc[i]) * weights['atr'].iloc[i]
        volume_score = (data['Volume'].iloc[i] > 1.5 * data['average_volume'].iloc[i]) * weights['vol'].iloc[i]
        ha_score = (data['ha_close'].iloc[i] > data['ha_open'].iloc[i]) * weights['ha'].iloc[i]

        score = macd_score + atr_score + volume_score + ha_score
        weighted_score.append(score)

    data['weighted_score'] = weighted_score
    return data

  data['Signal'] = 0
  processed_data = calculate_signals(data)
  processed_data = clean_repetitive_signals(processed_data)
  generate_tradetype(data)
  processed_data = processed_data[~(
      (processed_data['Signal'] == 0) &
      (processed_data['Close'] == 0) &
      (processed_data['Volume'] == 0)
  )]

  # Ensure last row is kept and Signal is set to 0
  processed_data.iloc[-1, processed_data.columns.get_loc('Signal')] = 0

  # After signal generation
  weights = calculate_dynamic_weights(processed_data)
  data = add_weighted_score(processed_data, weights)

  score_window = 55  # Lookback window
  k = 1.5  # Multiplier for standard deviation

  data['ovr_score_mean'] = data['weighted_score'].rolling(score_window).mean()
  data['ovr_score_std'] = data['weighted_score'].rolling(score_window).std()

  # Upper and lower dynamic thresholds
  data['dynamic_thresh_long'] = data['ovr_score_mean'] + k * data['ovr_score_std']
  data['dynamic_thresh_short'] = data['ovr_score_mean'] - (k - 1) * data['ovr_score_std'] #changed

  processed_data['dynamic_thresh_long'] = data['dynamic_thresh_long']
  processed_data['dynamic_thresh_short'] = data['dynamic_thresh_short']

  def siggen(signal_series, min_threshold, max_threshold, middle):

      position = 0
      signals = []

      for i in range(len(signal_series)):
          if position == 0:
              if signal_series.iloc[i] > max_threshold.iloc[i]:
                  position = -1
              elif signal_series.iloc[i] < min_threshold.iloc[i]:
                  position = 1
          elif position == -1:
              if signal_series.iloc[i] < min_threshold.iloc[i]:
                  position = 1
              elif signal_series.iloc[i] < middle.iloc[i]:
                  position = 0
          elif position == 1:
              if signal_series.iloc[i] > max_threshold.iloc[i]:
                  position = -1
              elif signal_series.iloc[i] > middle.iloc[i]:
                  position = 0
          signals.append(position)

      return pd.Series(signals, index=signal_series.index)

  processed_data['Signals'] = siggen(processed_data['weighted_score'], processed_data['dynamic_thresh_short'], processed_data['dynamic_thresh_long'], processed_data['ovr_score_mean'])

  data['trade'] = data['Signals'].map({1: 'short', -1: 'long'}).fillna('hold')

  processed_data['trade'] = data['trade']
  print(processed_data)

  # Get the last non-zero signal and its weighted score
  last_trade_row = processed_data[processed_data['Signal'] != 0].iloc[-1] if not processed_data[processed_data['Signal'] != 0].empty else None
  last_trade_signal = last_trade_row['Signal'] if last_trade_row is not None else 0
  last_trade_score = round(last_trade_row['weighted_score'], 4) if last_trade_row is not None else 0

  # Map to trade type
  signal_to_trade_type = {
      1: 'long',
      -1: 'short',
      0: 'hold'
  }
  latest_trade_type = signal_to_trade_type.get(last_trade_signal, 'hold')

  # Get the last long or short trade based on 'Signals' and 'trade'
  valid_trades = processed_data[(processed_data['Signals'] != 0) & (processed_data['trade'].isin(['long', 'short']))]

  if not valid_trades.empty:
      last_trade_row = valid_trades.iloc[-1]
      last_trade_signal = last_trade_row['Signals']
      latest_traded = last_trade_row['trade']
      last_trade_score = round(last_trade_row['weighted_score'], 4)
  else:
      last_trade_signal = 0
      latest_traded = 'hold'
      last_trade_score = 0

  new_type = processed_data['trade'].iloc[-2]
  curr_trade_type = processed_data['trade'].iloc[-1]

  # Write to G2 and G3
  worksheet.update('G2', [[latest_trade_type]])
  worksheet.update('G3', [[last_trade_score]])
  worksheet.update('G7', [[new_type]])
  worksheet.update('G8', [[curr_trade_type]])

  long_threshold = processed_data['dynamic_thresh_long'].dropna().iloc[-1]
  short_threshold = processed_data['dynamic_thresh_short'].dropna().iloc[-1]

  worksheet.update('G4', [[long_threshold]])
  worksheet.update('G5', [[short_threshold]])


  print(f"Trade type '{latest_trade_type}' written to G2 and score '{last_trade_score}' written to G3.")
  print(f"Long threshold '{long_threshold}' written to G4 and short threshold '{short_threshold}' written to G5.")


  processed_data.to_csv("processed_data_with_trades {}.csv".format(i), index=False)
  print("Processed data saved to 'processed_data_with_trades.csv'.")









                    Date     Open     High      Low    Close      Volume  \
0     5/25/2015 15:30:00   222.89   224.87   220.99   221.53   1512612.0   
1     5/26/2015 15:30:00   221.65   221.90   218.60   219.07   2173883.0   
2     5/27/2015 15:30:00   219.42   220.63   217.98   219.25   1846549.0   
3     5/28/2015 15:30:00   219.38   221.46   215.95   216.89   6066339.0   
4     5/29/2015 15:30:00   217.91   219.53   216.23   217.19   6692348.0   
...                  ...      ...      ...      ...      ...         ...   
2473   6/4/2025 15:30:00  1414.70  1425.70  1406.10  1423.50   8126827.0   
2474   6/5/2025 15:30:00  1428.00  1454.70  1423.60  1442.40  13716817.0   
2475   6/6/2025 15:30:00  1441.00  1451.50  1437.00  1443.50   8182161.0   
2476   6/9/2025 15:30:00  1457.00  1457.50  1445.30  1448.80   7885125.0   
2477  6/10/2025 15:30:00  1450.00  1453.90  1436.00  1438.50   7912897.0   

              Unnamed: 6  Open.1  High.1   Low.1  ...  atr_success  \
0                

<ipython-input-3-58c11b4753d4>:259: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G2', [[latest_trade_type]])
<ipython-input-3-58c11b4753d4>:260: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G3', [[last_trade_score]])
<ipython-input-3-58c11b4753d4>:261: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G7', [[new_type]])
<ipython-input-3-58c11b4753d4>:262: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G8', [[curr_trade_

Trade type 'short' written to G2 and score '0.35' written to G3.
Long threshold '2.1468462606283043' written to G4 and short threshold '0.8419603373663229' written to G5.
Processed data saved to 'processed_data_with_trades.csv'.
                    Date     Open     High      Low    Close     Volume  \
0     5/25/2015 15:30:00  1310.00  1318.45  1292.85  1300.50   491489.0   
1     5/26/2015 15:30:00  1302.50  1311.63  1294.45  1304.43   980870.0   
2     5/27/2015 15:30:00  1304.00  1316.58  1296.00  1307.63   810345.0   
3     5/28/2015 15:30:00  1301.95  1311.50  1291.13  1304.75  1381804.0   
4     5/29/2015 15:30:00  1308.50  1324.30  1298.63  1305.15  2211766.0   
...                  ...      ...      ...      ...      ...        ...   
2472   6/4/2025 15:30:00  3389.00  3394.80  3370.00  3380.90  2544807.0   
2473   6/5/2025 15:30:00  3409.00  3409.00  3363.70  3371.10  3584683.0   
2474   6/6/2025 15:30:00  3371.10  3390.90  3358.70  3384.20  2070390.0   
2475   6/9/2025 15:30

<ipython-input-3-58c11b4753d4>:259: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G2', [[latest_trade_type]])
<ipython-input-3-58c11b4753d4>:260: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G3', [[last_trade_score]])
<ipython-input-3-58c11b4753d4>:261: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G7', [[new_type]])
<ipython-input-3-58c11b4753d4>:262: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G8', [[curr_trade_

Trade type 'short' written to G2 and score '0.2' written to G3.
Long threshold '1.7879486235851019' written to G4 and short threshold '0.8052292466837538' written to G5.
Processed data saved to 'processed_data_with_trades.csv'.
                    Date     Open     High      Low    Close      Volume  \
0     5/25/2015 15:30:00   283.64   285.55   282.68   283.77   4982920.0   
1     5/26/2015 15:30:00   283.64   284.55   280.59   281.55   5005001.0   
2     5/27/2015 15:30:00   283.45   286.18   279.55   284.68  11415591.0   
3     5/28/2015 15:30:00   285.68   287.27   283.18   285.91  10472983.0   
4     5/29/2015 15:30:00   288.27   289.91   286.18   288.41  20604276.0   
...                  ...      ...      ...      ...      ...         ...   
2472   6/4/2025 15:30:00  1437.40  1440.10  1426.40  1431.00   8025388.0   
2473   6/5/2025 15:30:00  1430.90  1459.00  1427.30  1454.80  11385668.0   
2474   6/6/2025 15:30:00  1448.60  1461.40  1442.10  1459.70  11580994.0   
2475   6/9/2

...

Trade type 'short' written to G2 and score '0.15' written to G3.
Long threshold '2.0477201992616862' written to G4 and short threshold '0.7040932669127713' written to G5.
Processed data saved to 'processed_data_with_trades.csv'.
                    Date     Open     High      Low    Close      Volume  \
0     5/25/2015 15:30:00   355.84   368.54   353.49   360.34   5388695.0   
1     5/26/2015 15:30:00   362.23   366.70   358.81   360.93   6289299.0   
2     5/27/2015 15:30:00   360.57   372.24   360.57   368.32   5060958.0   
3     5/28/2015 15:30:00   371.25   371.92   360.39   363.14   7227151.0   
4     5/29/2015 15:30:00   362.91   391.97   362.51   383.50  67772622.0   
...                  ...      ...      ...      ...      ...         ...   
2471   6/4/2025 15:30:00  1855.00  1884.60  1850.00  1875.40   5892170.0   
2472   6/5/2025 15:30:00  1884.00  1889.00  1868.60  1878.80   8999675.0   
2473   6/6/2025 15:30:00  1872.10  1881.90  1867.30  1870.20   4644174.0   
2474   6/9/

<ipython-input-3-58c11b4753d4>:259: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G2', [[latest_trade_type]])
<ipython-input-3-58c11b4753d4>:260: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G3', [[last_trade_score]])
<ipython-input-3-58c11b4753d4>:261: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G7', [[new_type]])
<ipython-input-3-58c11b4753d4>:262: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G8', [[curr_trade_

Trade type 'short' written to G2 and score '0.5' written to G3.
Long threshold '1.9946781202679689' written to G4 and short threshold '0.8490466871834041' written to G5.
Processed data saved to 'processed_data_with_trades.csv'.
                    Date     Open     High      Low    Close     Volume  \
0     5/25/2015 15:30:00   268.10   269.95   264.95   266.95   673364.0   
1     5/26/2015 15:30:00   267.00   271.35   267.00   269.80   892044.0   
2     5/27/2015 15:30:00   269.50   273.90   268.60   270.95   880398.0   
3     5/28/2015 15:30:00   271.00   272.90   267.30   271.35   602620.0   
4     5/29/2015 15:30:00   272.45   276.50   270.70   275.75  1680703.0   
...                  ...      ...      ...      ...      ...        ...   
2468   6/4/2025 15:30:00  1502.20  1505.00  1492.00  1500.00   656425.0   
2469   6/5/2025 15:30:00  1505.50  1507.70  1482.00  1488.00  1934068.0   
2470   6/6/2025 15:30:00  1495.00  1526.00  1491.00  1523.20   707150.0   
2471   6/9/2025 15:30:

<ipython-input-3-58c11b4753d4>:259: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G2', [[latest_trade_type]])
<ipython-input-3-58c11b4753d4>:260: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G3', [[last_trade_score]])
<ipython-input-3-58c11b4753d4>:261: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G7', [[new_type]])
<ipython-input-3-58c11b4753d4>:262: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G8', [[curr_trade_

Trade type 'short' written to G2 and score '0.15' written to G3.
Long threshold '1.711287285670351' written to G4 and short threshold '0.6210860562917011' written to G5.
Processed data saved to 'processed_data_with_trades.csv'.


In [ ]:
import pandas as pd
from gspread_dataframe import get_as_dataframe

df = get_as_dataframe(worksheet, evaluate_formulas=True)
df.dropna(how='all', inplace=True)

df.to_csv("bajfinance_data.csv", index=False)


In [ ]:
#Eureka Moment!
import pandas as pd
import numpy as np
from ta.volatility import AverageTrueRange
from ta.trend import MACD
from ta.utils import dropna

data = pd.read_csv("/content/bajfinance_data.csv")
# Indicators
macd = MACD(data['Close'])
data['macd'] = macd.macd()
data['signal_line'] = macd.macd_signal()
data['macd_diff'] = macd.macd_diff()

atr = AverageTrueRange(data['High'], data['Low'], data['Close'])
data['atr'] = atr.average_true_range()

data['average_volume'] = data['Volume'].rolling(20).mean()

# Fibo Retract
data['fib_61.8'] = data['Close'] * 0.618

# HAC's
data['ha_close'] = (data['Open'] + data['Close'] + data['High'] + data['Low']) / 4
data['ha_open'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2
data['ha_open'] = data['ha_open'].fillna(data['ha_close'])


def calculate_signals(data, atr_threshold=0.015, volume_multiplier=1.5, indicator_threshold=2):
    data['Signal'] = 0
    data['position'] = 0

    current_position = 0

    for i in range(len(data)):
        score = 0

        macd_condition = data['macd_diff'].iloc[i] > 0
        atr_condition = data['atr'].iloc[i] > atr_threshold * data['Close'].iloc[i]
        volume_condition = data['Volume'].iloc[i] > volume_multiplier * data['average_volume'].iloc[i]
        fib_condition = abs(data['Close'].iloc[i] - data['fib_61.8'].iloc[i]) <= 0.01 * data['Close'].iloc[i]
        ha_condition_long = data['ha_close'].iloc[i] > data['ha_open'].iloc[i]
        ha_condition_short = data['ha_close'].iloc[i] < data['ha_open'].iloc[i]

        score += macd_condition
        score += atr_condition
        score += volume_condition
        score += fib_condition
        score += ha_condition_long or ha_condition_short

        raw_signal = 0
        if score >= indicator_threshold:
            if macd_condition and ha_condition_long:
                raw_signal = 1
            elif macd_condition and ha_condition_short:
                raw_signal = -1

        # Now decide the actual signal
        signal = 0

        if current_position == 0:
            if raw_signal == 1:
                signal = 1
                current_position = 1
            elif raw_signal == -1:
                signal = -1
                current_position = -1
        elif current_position == 1:
            if raw_signal == -1:
                signal = -1  # reverse directly to short
                current_position = -1
            elif raw_signal != 1:
                signal = 0  # square off or hold
        elif current_position == -1:
            if raw_signal == 1:
                signal = 1  # reverse directly to long
                current_position = 1
            elif raw_signal != -1:
                signal = 0  # square off or hold


        data.at[i, 'Signal'] = signal
        data.at[i, 'position'] = current_position  # <-- Save position history

    return data


def clean_repetitive_signals(data):
    previous_signal = 0
    for i in range(len(data)):
        current_signal = data.at[i, 'Signal']


        if current_signal != 0 and current_signal == previous_signal:
            data.at[i, 'Signal'] = 0
        elif current_signal != 0:
            previous_signal = current_signal

    return data



def generate_tradetype(data):

    data['trade_type'] = data['Signal'].map({
        2: 'long',
        1: 'long',
        0: 'hold',
        -1: 'short',
        -2: 'short'
    })

    return data


data['Signal'] = 0
processed_data = calculate_signals(data)
processed_data = clean_repetitive_signals(processed_data)
generate_tradetype(data)
processed_data = processed_data[~(
    (processed_data['Signal'] == 0) &
    (processed_data['Close'] == 0) &
    (processed_data['Volume'] == 0)
)]

# Ensure last row is kept and Signal is set to 0
processed_data.iloc[-1, processed_data.columns.get_loc('Signal')] = 0

print(processed_data)

# Get the latest trade_type from the last row
latest_trade_type = processed_data['trade_type'].iloc[-1]

# Write it back to G2 in the same worksheet
worksheet.update('G2', [[latest_trade_type]])


print(f"Trade type '{latest_trade_type}' written to G2 in the original Google Sheet.")


processed_data.to_csv("processed_data_with_trades.csv", index=False)
print("Processed data saved to 'processed_data_with_trades.csv'.")






                    Date    Open    High     Low   Close      Volume  \
0     5/26/2020 15:30:00  190.00  194.95  187.80  191.70  53235758.0   
1     5/27/2020 15:30:00  192.50  193.90  186.90  192.15  24546312.0   
2     5/28/2020 15:30:00  192.30  192.90  189.00  190.65  24579385.0   
3     5/29/2020 15:30:00  190.45  198.00  189.00  197.35  35836696.0   
4      6/1/2020 15:30:00  201.00  205.25  199.00  200.55  28509025.0   
...                  ...     ...     ...     ...     ...         ...   
1230  5/19/2025 15:30:00  435.00  438.50  432.65  435.35   7543854.0   
1231  5/20/2025 15:30:00  437.00  440.80  432.90  434.90   9868414.0   
1232  5/21/2025 15:30:00  436.60  438.20  432.00  433.00   5776672.0   
1233  5/22/2025 15:30:00  433.00  433.00  422.90  426.10  19451749.0   
1234  5/23/2025 15:30:00  434.00  439.00  429.00  436.30  18536791.0   

          macd  signal_line  macd_diff       atr  average_volume  fib_61.8  \
0          NaN          NaN        NaN  0.000000         

<ipython-input-10-91e49c81507f>:135: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('G2', [[latest_trade_type]])


Trade type 'hold' written to G2 in the original Google Sheet.
Processed data saved to 'processed_data_with_trades.csv'.
